# This notebook shows how to use Nvidia Retriever Microservices as a LangChain retriever.
Note, for this notebook to run, you will have to spin up the NVIDIA Retriever Microservices by following the instruction from this [gitlab repo ](https://gitlab-master.nvidia.com/dl/ai-services/microservices/retrieval/-/blob/main/docs/developer_getting_started.md)

referring to https://python.langchain.com/docs/integrations/retrievers/weaviate-hybrid

## first , we will initiate the NvidiaRetrieverMicroservice 

In [30]:
import langchain_community
from langchain_community.retrievers.nv_retriever_microservices import NvidiaRetrieverMicroservice
# pipeline to choose from : dense_pgvector, dense_milvus, dense_elasticsearch , test_dense_milvus_random
collection_name="test1"
pipeline="dense_pgvector"
endpoint_url='http://localhost:1984/v1/collections'
collection_id='abc' # we give a dummy id for initialization, we will get a real collection_id once the service is successfully intialized 
NVretriever=NvidiaRetrieverMicroservice(collection_name,pipeline,endpoint_url, collection_id)


In [34]:
NVretriever.collection_id


'badc5da6-e57c-4e08-acb0-e7c7291aaa36'

## then we will set up a specific DocumentLoader which will allow us to access each file path

In [31]:
from langchain_community.document_loaders.filepathloader import BasicFilePathLoader
folder_dir="/workspace/data/"
loader=BasicFilePathLoader(folder_dir)

In [33]:
docs=loader.load()
for doc in docs:
    print(doc.page_content, doc.metadata)

Sweden.txt {'source': '/workspace/data/Sweden.txt'}
Titanic_film.txt {'source': '/workspace/data/Titanic_film.txt'}


## we then upload these documents to Nvidia Retriever Microservices

In [20]:
NVretriever.add_documents(docs)

<class 'int'>
<class 'int'>


['46f2d7e8886a32c23f2a95284bd7147f194daa184973202dcdf5168f011caa16',
 '536174152c9874cef428dc503280a64af9f002db48130ec19443ccaf46a03e24']

## then we are ready to query 

In [26]:
out=NVretriever.get_relevant_documents("who directed the film Titanic?")

200 <class 'int'>


## note that score is contained in metadata by default, you can adjust filtering using score threshold

In [29]:
for o in out:
    if o.metadata['metadata']['score'] >=0.15:
        print(o.page_content)
        print('...'*20)

sheer scale and tragedy" and that "when you consider that [the film] tops a bum-numbing three-hour running time, then you have a truly impressive feat of entertainment achieved by Cameron". Empire eventually reinstated its original five-star rating of the film, commenting: "It should be no surprise[,] then[,] that it became fashionable to bash James Cameron's Titanic at approximately the same time it became clear that this was the planet's favourite film. Ever."In 2017, on the 20th anniversary of its release, the film was selected for preservation in the United States National Film Registry by the Library of Congress as being "culturally, historically, or aesthetically significant". It was listed among the 100 best films in an Empire poll and in a later poll of members of the film industry. In 2021, Dalin Rowell of /Film ranked it the third-best film of Cameron's career, stating that it is "easily one of his best films, simply because it defied the odds", and considering it "a legitima

In [36]:
import getpass
import os

## API Key can be found by going to NVIDIA NGC -> AI Foundation Models -> (some model) -> Get API Code or similar.
## 10K free queries to any endpoint (which is a lot actually).

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key


NVAPI Key (starts with nvapi-):  ······································································


In [41]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.text_splitter import CharacterTextSplitter
from langchain_nvidia_ai_endpoints import ChatNVIDIA


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

model = ChatNVIDIA(model="mixtral_8x7b")

chain = (
    {"context": NVretriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the plot of the film Titanic made in 1997, what is the name of the research vessel?")

200 <class 'int'>


'In the 1997 film Titanic, the research vessel is named "AKademik Mstislav Keldysh." It is depicted as the ship from which the wreck of the Titanic is explored. The real-life AKademik Mstislav Keldysh is a Russian research vessel that has been involved in numerous expeditions to the actual Titanic wreckage.'